Load the data

In [ ]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
import seaborn as snn

In [ ]:
mck = pd.read_csv('CAX_McK.csv')

In [ ]:
mck.head()

In [ ]:
# Summary of the data

mck.info()

In [ ]:
mck.describe()

In [ ]:
# Find the Nas in the dataset

mck.isna().any()
mck.isna().sum()

In [ ]:
# Create New variable duration_speed = distance_km / duration_min

# mck.assign(duration_speed = mck.distance_km / mck.duration_min)


# mck["duration_speed"] = mck["distance_km"] / mck["duration_min"]

In [ ]:
# Check the coloumns in dataframe

print(mck.columns)

Analysis on driver_response variable which is target variable

In [ ]:
# % response of the driver 1 mean accept &  0 mean decline

mck['driver_response'].value_counts(normalize=True) * 100

In [ ]:
# % of driver response based on unique driver code in decending order

# Create 'Sub-groups' for driver accept offer and driver reject the offer

driver_accept = mck[mck["driver_response"] == 1]

driver_reject = mck[mck["driver_response"] == 0]

In [ ]:
# Here not very clear..  which driver is accepting more and vis-versa

mck.groupby('driver_gk')['driver_response'].value_counts()[1:20]

In [ ]:
# Find the top 10 drivers who are acception the rides

driver_accept['driver_gk'].value_counts()[1:10]

In [ ]:
# Find the top 10 drivers who rejects the rides

driver_reject['driver_gk'].value_counts()[1:10]

In [ ]:
# Week Days wise, Driver Response rate.. Very low variation in weekdays. But, yes Wedensay has highest acceptance rate. 

mck.groupby('weekday_key')['driver_response'].value_counts(normalize = True) * 100

Outlier Study 

In [ ]:
mck[['distance_km', 'duration_min']].describe()

In [ ]:
# Box Plot to identify the outliers 

plt.subplot(2,1,2)
snn.boxplot(y = 'distance_km',  x = 'driver_response', data = mck)
plt.title('Distance in Km')
plt.show()

plt.subplot(2,1,2)
snn.boxplot(y = 'duration_min', x = 'driver_response', data = mck)
plt.title('Duration Time in Min')
plt.show()


 Remove the Outliers with Percentile method

In [ ]:
# Also remove the -1 from variables like especially from latitude and longitude

mck = mck[ (mck['driver_latitude'] > 0) & 
          (mck['driver_longitude'] > 0) &
          (mck['origin_order_latitude'] > 0) & 
          (mck['origin_order_longitude'] > 0) &
          (mck['distance_km'] > 0) &
          (mck['duration_min'] > 0) ] 

In [ ]:

plt.figure(figsize = (8,7))
plt.subplot(2,1,2)
snn.boxplot(y = 'distance_km', data = mck)
plt.show()


plt.figure(figsize = (10,7))
plt.subplot(2,1,2)
snn.boxplot(y = 'duration_min', x = 'driver_response', data = mck)
plt.show() 

## Still many outliers recides in these variables 

In [ ]:
# Check the Summary

mck[['driver_latitude', 'driver_longitude', 'origin_order_latitude', 'origin_order_longitude', 
    'distance_km', 'duration_min']].describe()

In [ ]:
# Outlier baed on Percentile method

np.quantile(mck['distance_km'], 0.99)

In [ ]:
np.quantile(mck['duration_min'], 0.99)

In [ ]:
# NOTE:  # Sharp rise in km and min [both], at percentile change from 0.99 to 0.99. hence, decided to take 0.998 

In [ ]:
mck = mck[ (mck['distance_km'] <= 72.87) & (mck['duration_min'] <= 72.63) ]

Add New Varianle - speeed

In [ ]:
# mck["duration_speed"] = mck["distance_km"] / mck["duration_min"]

# We decided not to add.. as it could create Multicolinarity.  

Correlation study

In [ ]:
mck.corr()

In [ ]:
# Correlation heatmap  

plt.figure(figsize=(15,5))

# Use 'mask' to hide duplicate boxes

Mask = np.triu(np.ones_like(mck.corr(), dtype = np.bool)) 

heatmap = snn.heatmap(mck.corr(), vmin = -1, vmax = 1,
           annot = True, cmap='YlGn', mask = Mask)


# Set Title
heatmap.set_title('Correlation heatmap', fontdict = {'fontsize':15}, pad=10);

plt.show()

In [ ]:
# NOTE: DO not directly remove the higl corr variables.. like origh and driver lattitude.. this means 
# If drive and orighn lattitude is high .. chances that driver will reject offer..  

# Can we play and create new features based on drivet (lat, long) and origin (lat, long)

In [ ]:
# Feature Creation :: Haversine Distance Between 

# driver_latitude & driver_longitude = driver_point

# origin_order_latitude & origin_order_longitude = origin_point


driver_latitude, driver_longitude, origin_order_latitude, origin_order_longitude = map(np.radians, (mck['driver_latitude'], mck['driver_longitude'], mck['origin_order_latitude'], mck['origin_order_longitude']))
    
AVG_EARTH_RADIUS = 6371  # in km
    
lat = origin_order_latitude - driver_latitude
lng = origin_order_longitude - driver_longitude

d = np.sin(lat * 0.5) ** 2 + np.cos(driver_latitude) * np.cos(origin_order_latitude) * np.sin(lng * 0.5) ** 2
mck['driver_origin_dist'] = 2 * AVG_EARTH_RADIUS * np.arcsin(np.sqrt(d))

In [ ]:
# Check Outliers into New Feature : driver_origin_dist

snn.boxplot(y = 'driver_origin_dist', x = 'driver_response', data = mck)
plt.show() 

In [ ]:
permax = np.quantile(mck['driver_origin_dist'], 0.999)
permin = np.quantile(mck['driver_origin_dist'], 0.01)


print("99.9% percentile Area covers :: ", permax)
print("0.01% percentile Area covers :: ", permin)



In [ ]:
# Indicated that 99.9% data is under 6.15 km. And hence decided to trim driver_origin_dist

mck = mck[ (mck['driver_origin_dist'] <= 6.1547) & 
          (mck['driver_origin_dist'] >= 0.01061710)]

In [ ]:
# ONly correlation of driver_response with all other variables

heatmap = snn.heatmap(mck.corr()[['driver_response']], vmin = -1, vmax = 1, annot = True, cmap='BrBG')

# As we can see first 5 featurs has very low corr.. However, those are Discrete variables. 

In [ ]:
mck.head()

In [ ]:
# In driver_gk and order_gk .. We can see significant diffrence between Driver Responce, But offer_gk is neutral for 0 and 1. 

mck.groupby('driver_response')['driver_gk'].value_counts(normalize = True) * 100

Statistica Test for Significance Test

In [ ]:
# Apply Chi Square Test and chek the Significant level

from scipy.stats import chi2_contingency

In [ ]:
# Forst convert Int to str

chi1 = mck[['driver_response', 'driver_gk']].astype(str)    # 6823 unique out of 628306 And hence significant, with highest frequency of 2333

chi2 = mck[['driver_response', 'order_gk']].astype(str)     # 501742 unique out of 628306 And hence not very significant, with less frequency of 27

chi3 = mck[['driver_response', 'offer_gk']].astype(str)     # All Unique and hence not significant, with least frequency of 1

In [ ]:
chi1.describe()

In [ ]:
mck.groupby('driver_gk')['driver_response'].value_counts(normalize = True) * 100

# 6821 driver_gk has 100% score in accept thr ride.. so we can say when request goes to 6821.. request will be accepted!

Statistical Test for variables

In [ ]:
# T-test for Continious variables with respect to driver_responce
# significant threashold level set at 95% 

from scipy import stats

import researchpy as rp

In [ ]:
rp.ttest(mck['distance_km'], mck['driver_response'], paired=True)

In [ ]:
dec, rec = rp.ttest(mck['distance_km'], mck['driver_response'], paired=True)

In [ ]:
dec

In [ ]:
# Remove all latitude and longitude features.. Plus Also remove ..  offer_gk

mck = mck.drop(['driver_latitude', 'driver_longitude', 'origin_order_latitude', 'origin_order_longitude',
                'order_gk', 'offer_gk'], axis = 1)

NOrmalization 

In [ ]:
from scipy.stats import boxcox

# REFER :: https://www.statology.org/box-cox-transformation-python/

# REFER :: https://towardsdatascience.com/box-cox-transformation-explained-51d745e34203

In [ ]:

plt.subplot(1,3,1)
snn.distplot(mck['distance_km'], hist=False, kde=True)


plt.subplot(1,3,2)
snn.distplot(mck['duration_min'], hist=False, kde=True)


plt.subplot(1,3,3)
snn.distplot(mck['driver_origin_dist'], hist=False, kde=True)


plt.show()



In [ ]:
# BoxCox Lambda test 

mck['distance_km'], best_lambda1 = boxcox(mck['distance_km'])

mck['duration_min'], best_lambda2 = boxcox(mck['duration_min'])

mck['driver_origin_dist'], best_lambda = boxcox(mck['driver_origin_dist'])

In [ ]:
print('Best Lambda 1:: ', best_lambda1)

print('Best Lambda 2:: ', best_lambda2)

print('Best Lambda 3:: ', best_lambda3)

In [ ]:
mck.describe()

Write the csv file

In [ ]:
mck.to_csv('Model_Mck.csv')

In [ ]:
## Build Supervised ML model on EDA_Mck.csv